# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:
cities_df = pd.read_csv('../output_data/city_data.csv')
cities_df = cities_df.drop(columns=['Unnamed: 0'])
cities_df.head()

,city,Cloundiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,jijiga,35,ET,1665350250,70,9.3500,42.8000,64.13,4.45
1,ushuaia,40,AR,1665349883,81,-54.8000,-68.3000,44.26,14.97
2,lakes entrance,7,AU,1665350251,76,-37.8811,147.9810,47.16,6.04
3,hobart,40,AU,1665349971,81,-42.8794,147.3294,49.05,10.36
4,kokkola,0,FI,1665350127,96,63.8385,23.1307,46.92,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
#configure gmaps
gmaps.configure(api_key=g_key)

#get locations and humidity
city_locations = cities_df[['Latitude', 'Longitude']].astype(float)
humidity = cities_df['Humidity'].astype(float)

#add heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
ideal_weather = cities_df.loc[(cities_df['Max Temp'] > 70) & (cities_df['Max Temp'] < 80) & 
                                  (cities_df['Wind Speed'] < 10) & (cities_df['Cloundiness'] == 0)]
ideal_weather

,city,Cloundiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
6,buin,0,CL,1665350251,22,-33.7333,-70.7500,79.12,6.42
176,worland,0,US,1665350281,29,44.0169,-107.9554,75.67,6.91
178,russell,0,US,1665350281,20,32.3502,-85.1999,76.15,6.91
232,beloha,0,MG,1665350292,87,-25.1667,45.0500,71.01,6.42
270,bani walid,0,LY,1665350297,63,31.7566,13.9942,70.93,9.98
288,shakawe,0,BW,1665350300,16,-18.3667,21.8500,72.46,7.34
295,riyadh,0,SA,1665349898,14,24.6877,46.7219,77.14,7.34
326,gat,0,IL,1665350062,62,31.6100,34.7642,70.48,3.42
365,waddan,0,LY,1665350311,36,29.1614,16.1390,73.56,8.25
411,marovoay,0,MG,1665350319,57,-16.1000,46.6333,72.70,7.54


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df

,city,Cloundiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,buin,0,CL,1665350251,22,-33.7333,-70.7500,79.12,6.42
1,worland,0,US,1665350281,29,44.0169,-107.9554,75.67,6.91
2,russell,0,US,1665350281,20,32.3502,-85.1999,76.15,6.91
3,beloha,0,MG,1665350292,87,-25.1667,45.0500,71.01,6.42
4,bani walid,0,LY,1665350297,63,31.7566,13.9942,70.93,9.98
5,shakawe,0,BW,1665350300,16,-18.3667,21.8500,72.46,7.34
6,riyadh,0,SA,1665349898,14,24.6877,46.7219,77.14,7.34
7,gat,0,IL,1665350062,62,31.6100,34.7642,70.48,3.42
8,waddan,0,LY,1665350311,36,29.1614,16.1390,73.56,8.25
9,marovoay,0,MG,1665350319,57,-16.1000,46.6333,72.70,7.54


In [41]:
hotel_df['Hotel Name'] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#declearing the parameter 
params  = {
    "radius": 50000,
    "rankby": "prominence",
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,
}

#creating counter varibale
counter = 1
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get variables type from df
    lat = row['Latitude']
    lng = row['Longitude']
    city = row['city']
    params["location"] = f'{lat},{lng}'
        
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel for {counter}. {city} is {results[0]['name']}.")
        counter = counter + 1
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except:
        print(f"Missing for {city}... skipping.")

Closest hotel for 1. buin is Holiday Inn Santiago - Airport Terminal, an IHG Hotel.
Closest hotel for 2. worland is Best Western Plus Plaza Hotel.
Closest hotel for 3. russell is Holiday Inn Express Abrams Hall On Fort Benning.
Missing for beloha... skipping.
Closest hotel for 4. bani walid is فندق التاج.
Closest hotel for 5. shakawe is Shametu River Lodge & Campsite.
Closest hotel for 6. riyadh is Super 8 by Wyndham Al Riyadh.
Closest hotel for 7. gat is Sadot Hotel.
Closest hotel for 8. waddan is Al Nakhel Hotel.
Closest hotel for 9. marovoay is Baobab Tree Hotel & Spa.
Closest hotel for 10. ada is Homewood Suites by Hilton Eagle Boise.
Closest hotel for 11. tukrah is Tibesti Hotel.
Closest hotel for 12. jalu is إستراحة صلاح الصادق.


In [47]:
hotel_df = hotel_df[hotel_df['Hotel Name'] != ""]
hotel_df = hotel_df.rename(columns={"Latitude": "Lat", "Longitude": "Lng"})
hotel_df

,city,Cloundiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,buin,0,CL,1665350251,22,-33.7333,-70.7500,79.12,6.42,"Holiday Inn Santiago - Airport Terminal, an IH..."
1,worland,0,US,1665350281,29,44.0169,-107.9554,75.67,6.91,Best Western Plus Plaza Hotel
2,russell,0,US,1665350281,20,32.3502,-85.1999,76.15,6.91,Holiday Inn Express Abrams Hall On Fort Benning
4,bani walid,0,LY,1665350297,63,31.7566,13.9942,70.93,9.98,فندق التاج
5,shakawe,0,BW,1665350300,16,-18.3667,21.8500,72.46,7.34,Shametu River Lodge & Campsite
6,riyadh,0,SA,1665349898,14,24.6877,46.7219,77.14,7.34,Super 8 by Wyndham Al Riyadh
7,gat,0,IL,1665350062,62,31.6100,34.7642,70.48,3.42,Sadot Hotel
8,waddan,0,LY,1665350311,36,29.1614,16.1390,73.56,8.25,Al Nakhel Hotel
9,marovoay,0,MG,1665350319,57,-16.1000,46.6333,72.70,7.54,Baobab Tree Hotel & Spa
10,ada,0,US,1665350337,27,43.4166,-116.1262,76.23,8.01,Homewood Suites by Hilton Eagle Boise


In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [50]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))